In [ ]:
import requests
import polars as pl

In [ ]:
# This url DOES NOT have match results
# We don't want to be pulling data from this all the time
# because it is a lot of data and it is not needed
# The idea is to store this in an olap and pull analysis to display in a UI
url = 'https://sportapi.mlssoccer.com/api/matches?competition=98&excludeSecondaryTeams=true&excludeVenue=false'


def get_matches():
    """Get the list of matches from the MLS API."""
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Failed to fetch data: {response.status_code}")
    return response.json()

data = get_matches()
    

In [ ]:
# Flatten the nested dictionary and rename columns to avoid conflicts
match_data = []
dim_teams = []


for item in data:
    match = {
        'Match_SK': item['optaId'],
        'HomeTeamKey': item['home']['optaId'],
        'AwayTeamKey': item['away']['optaId'],
        'VenueKey': item['venue']['venueSportecId'],
        'SeasonKey': item['season']['slug'],
        'CompetitionKey': item['season']['competitionId'],
        'Match_DateTime': item['matchDate'],
        'isDelayed': item['delayedMatch'],
        'MatchDay': item['matchDay']
    }
    
    team_home = {
        'Team_SK': item['home']['optaId'],
        'TeamName': item['home']['fullName'],
        'TeamAbbreviation': item['home']['abbreviation'],
        'TeamLogo': item['home']['logoColorUrl'].replace('{formatInstructions}', 'w_200')
    }
    
    team_away = {
        'Team_SK': item['away']['optaId'],
        'TeamName': item['away']['fullName'],
        'TeamAbbreviation': item['away']['abbreviation'],
        'TeamLogo': item['away']['logoColorUrl'].replace('{formatInstructions}', 'w_200')
    }
    
    match_data.append(match)
    dim_teams.append(team_home)
    dim_teams.append(team_away)
    




### Matches

In [ ]:
fct_match = pl.DataFrame(match_data)
fct_match = fct_match.with_columns(pl.col("Match_DateTime").str.strptime(pl.Datetime, "%Y-%m-%dT%H:%M:%S.%fZ"))


### Dim Teams

In [ ]:
# Deduplicate the list of dictionaries based on 'Team_SK'
dim_teams = list({team['Team_SK']: team for team in dim_teams}.values())

dim_teams